In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# R code 8.19

d = pd.read_csv('Data/tulips.csv', sep=';', header=0)

In [3]:
# R code 8.20
d['blooms_std']= d.blooms/d.blooms.max()
d['water_cent']=d.water - d.water.mean()
d['shade_cent']=d.shade -d.shade.mean()

In [4]:
# R code 8.24

with pm.Model() as m_8_7:
    a = pm.Normal('a', mu=0.5, sd=0.25) 
    b_water = pm.Normal('b_water', mu=0, sd=0.25) 
    b_shade = pm.Normal('b_shade', mu=0, sd=0.25)
    b_water_shade = pm.Normal('b_water_shade', mu=0, sd=0.25)
    mu = (a + b_water * (d.water_cent) + b_shade * d.shade_cent 
          + b_water_shade * d.water_cent * d.shade_cent)
    sigma = pm.Exponential('sigma', 1)
    blooms_std = pm.Normal('blooms_std', mu=mu, sd=sigma, observed=d.blooms_std)
    trace_8_7= pm.sample(5000, tune=1000, chains=1)
    prior_8_7 = pm.sample_prior_predictive(samples=1000)
    #start = {'a':np.mean(d.blooms), 'b_water':0, 'b_shade':0, 'b_water_shade':0,'sigma':np.std(d.blooms)}

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sigma, b_water_shade, b_shade, b_water, a]
100%|████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:05<00:00, 1151.68it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


In [ ]:
bid = pd.Categorical(d['bed'])
d['bid']=bid.codes

with pm.Model() as m_8_8:
    a = pm.Normal('a', mu=0.5, sd=0.25, shape=2) 
#     b_water = pm.Normal('b_water', mu=0, sd=0.25)#, shape=len(bid.categories)) 
#     b_shade = pm.Normal('b_shade', mu=0, sd=0.25)#, shape=len(bid.categories))
#     b_water_shade = pm.Normal('b_water_shade', mu=0, sd=0.25)#, shape=len(bid.categories))
    mu = (a[(bid)])
    sigma = pm.Exponential('sigma', 1)
    blooms_std = pm.Normal('blooms_std', mu=mu, sd=sigma, observed=d.blooms_std)
    trace_8_8= pm.sample(5000, tune=1000, chains=1)
    #prior_8_8 = pm.sample_prior_predictive(samples=1000)